In [1]:
import os, joblib
import xgboost as xgb
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GroupShuffleSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.utils import resample
# from imblearn.over_sampling import SMOTE
# from imblearn.ensemble import BalancedRandomForestClassifier

/home/administrator/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Define Data directory
DATA_DIR = '../data/'

# list of poet names
# Removed the 5 lowest poets
poets = ["abbasinYousuf", "ajmalKhattak", "allamaAbdulHai", "ghaniKhan", "hamzaBaba", "javedAhmedzai", "karanKhan", "khatirAfridi", "khushalKhanKhattak", "matiullahTurab", "mumtazOrakazi", "munirJan", "naeemAhmed", "rabiaMumtaz", "rahmanBaba", "rehmatShah", "sahibShahSabir", "shabbirKhanDurrani", "shakirOrakzai", "shoaibKhanKhattak"]

# # Pehle k 13
# poets = ["abbasinYousuf", "azizMazerwal", "ghaniKhan", "hamzaBaba", "khaliqZiari", "khatirAfridi", "khushalKhanKhattak",  "mumtazOrakazi", "munirJan",  "rahmanBaba", "rehmatShah", "sahibShahSabir", "salimRiaz"]

## All
# poets = ["abbasinYousuf", "azizMazerwal", "ghaniKhan", "hamzaBaba", "karanKhan", "khaliqZiari", "khatirAfridi", "khushalKhanKhattak", "matiullahTurab", "mumtazOrakazi", "munirJan", "naeemAhmed", "rabiaMumtaz", "rahmanBaba", "rehmatShah", "sahibShahSabir", "salimRiaz"]

poet_labels = {poet: i for i, poet in enumerate(poets)}
print(poet_labels)

{'abbasinYousuf': 0, 'ajmalKhattak': 1, 'allamaAbdulHai': 2, 'ghaniKhan': 3, 'hamzaBaba': 4, 'javedAhmedzai': 5, 'karanKhan': 6, 'khatirAfridi': 7, 'khushalKhanKhattak': 8, 'matiullahTurab': 9, 'mumtazOrakazi': 10, 'munirJan': 11, 'naeemAhmed': 12, 'rabiaMumtaz': 13, 'rahmanBaba': 14, 'rehmatShah': 15, 'sahibShahSabir': 16, 'shabbirKhanDurrani': 17, 'shakirOrakzai': 18, 'shoaibKhanKhattak': 19}


In [3]:
# Load and Label the Data
def load_and_label(data_dir, poets, poet_labels):
    data = []
    labels = []

    for poet in poets:
        poet_dir = os.path.join(data_dir, poet)
        file_path = os.path.join(poet_dir, f'{poet}.txt')

        # Check if the file exists
        if not os.path.exists(file_path):
            print(f'{file_path} does not exist')
            continue

        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            data.extend(lines)
            labels.extend([poet_labels[poet]] * len(lines))

    df = pd.DataFrame({'text': data, 'label': labels})
    return data, labels, df

In [4]:
data, labels, df = load_and_label(DATA_DIR, poets, poet_labels)
df.describe()

,label
count,54620.000000
mean,8.625265
std,5.352617
min,0.000000
25%,4.000000
50%,8.000000
75%,14.000000
max,19.000000


In [5]:
def balance_data(df):
    max_size = df['label'].value_counts().max()
    lst = [df]
    for class_index, group in df.groupby('label'):
        lst.append(group.sample(max_size-len(group), replace=True))
    df_balanced = pd.concat(lst)
    return df_balanced.sample(frac=1).reset_index(drop=True)

In [6]:
df_balanced = balance_data(df)

In [ ]:
def weighted_train_test_split(df, test_size=0.2, random_state=42):
    train_dfs = []
    test_dfs = []

    # Split each poet's data individually
    for poet_label in df['label'].unique():
        poet_df = df[df['label'] == poet_label]
        train_poet, test_poet = train_test_split(
            poet_df, test_size=test_size, random_state=random_state, stratify=poet_df['label']
        )
        train_dfs.append(train_poet)
        test_dfs.append(test_poet)

    # Concatenate all individual splits to get the final training and testing dataframes
    train_df = pd.concat(train_dfs).sample(frac=1, random_state=random_state).reset_index(drop=True)
    test_df = pd.concat(test_dfs).sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    return train_df, test_df

In [ ]:
# Modified extract_features function
def extract_features(df, tfidf_vectorizer=None, num_features=None):
    if tfidf_vectorizer is None:
        tfidf_vectorizer = TfidfVectorizer(max_features=num_features)
        X = tfidf_vectorizer.fit_transform(df['text']).toarray()
    else:
        X = tfidf_vectorizer.transform(df['text']).toarray()
    y = df['label'].values
    return X, y, tfidf_vectorizer

In [7]:
def extract_features(df, num_features=None):
    tfidf_vectorizer = TfidfVectorizer(max_features=num_features)
    # tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X = tfidf_vectorizer.fit_transform(df['text']).toarray()
    y = df['label'].values
    return X, y, tfidf_vectorizer

In [ ]:
# train_df, test_df = weighted_train_test_split(df, test_size=0.2)
# X_train, y_train, tfidf_vectorizer = extract_features(train_df, 5000)
# X_test, y_test, _ = extract_features(test_df, 5000)
# train_df, test_df = weighted_train_test_split(df_balanced, test_size=0.2)
train_df, test_df = weighted_train_test_split(df, test_size=0.2)
X_train, y_train, tfidf_vectorizer = extract_features(train_df, num_features=10000)  # Fit vectorizer on training data
X_test, y_test, _ = extract_features(test_df, tfidf_vectorizer=tfidf_vectorizer)  # Use same vectorizer for test data
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Check the split results
print(f"Training set size: {len(train_df)} samples")
print(f"Testing set size: {len(test_df)} samples")
print(f"Class distribution in training set:\n{train_df['label'].value_counts(normalize=True)}")
print(f"Class distribution in testing set:\n{test_df['label'].value_counts(normalize=True)}")

In [ ]:
# smote = SMOTE(random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [8]:
# X, y, tfidf_vectorizer = extract_features(df)
# X, y, tfidf_vectorizer = extract_features(df, 15000)
# X, y, tfidf_vectorizer = extract_features(df, 5000)
X, y, tfidf_vectorizer = extract_features(df_balanced, 5000)

In [10]:
# print(f"Number of samples: {len(df)}")
# print(f"Number of features (TF-IDF): {X.shape[1]}")
# print(f"Class distribution:\n{df['label'].value_counts()}")
print(f"Number of samples: {len(df_balanced)}")
print(f"Number of features (TF-IDF): {X.shape[1]}")
print(f"Class distribution:\n{df_balanced['label'].value_counts()}")

Number of samples: 170280
Number of features (TF-IDF): 5000
Class distribution:
label
15    8514
19    8514
9     8514
11    8514
16    8514
17    8514
18    8514
4     8514
1     8514
7     8514
3     8514
5     8514
10    8514
6     8514
14    8514
12    8514
13    8514
0     8514
2     8514
8     8514
Name: count, dtype: int64


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((136224, 5000), (34056, 5000), (136224,), (34056,))

#### Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
# rf = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)

print(f"Training the Random Forest model...")
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
print(f"Training and Predicting Done!")

Training the Random Forest model...


In [ ]:
# Evaluation
accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
conf_mat_rf = confusion_matrix(y_test, y_pred_rf)

# Print Metrics
print("\nModel Performance and Evaluation")
# print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Accuracy: {accuracy_rf*100:.2f}%")
print(f"F1 Score (Weighted): {f1_rf:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_rf)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_rf, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Random Forest')
plt.show()

#### Support Vector Machine

In [ ]:
# SVM Model
svm = SVC(kernel='linear', random_state=42)

# Training and predictions
print(f"Training the SVM model...")
svm.fit(X_train, y_train)

y_pred_svm = svm.predict(X_test)
print(f"Training and Predicting Done!")

In [ ]:
# Evaluation
accuracy_svm = accuracy_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')
conf_mat_svm = confusion_matrix(y_test, y_pred_svm)

# Print Metrics
print("\nModel Performance and Evaluation")
# print(f"Accuracy: {accuracy_svm:.4f}")
print(f"Accuracy: {accuracy_svm*100:.2f}%")
print(f"F1 Score (Weighted): {f1_svm:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_svm)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_svm, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - SVM')
plt.show()

In [ ]:
# SVM Model
svm = SVC(C=1.0, kernel='rbf', gamma='scale', probability=True, random_state=42)

# Training and predictions
print(f"Training the SVM model...")
svm.fit(X_train, y_train)

y_pred_svm = svm.predict(X_test)
print(f"Training and Predicting Done!")

In [ ]:
# Evaluation
accuracy_svm = accuracy_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')
conf_mat_svm = confusion_matrix(y_test, y_pred_svm)

# Print Metrics
print("\nModel Performance and Evaluation")
# print(f"Accuracy: {accuracy_svm:.4f}")
print(f"Accuracy: {accuracy_svm*100:.2f}%")
print(f"F1 Score (Weighted): {f1_svm:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_svm)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_svm, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - SVM')
plt.show()

#### Logistic Regression

In [ ]:
log_reg = LogisticRegression(max_iter=1000)

print(f"Training the Logistic Regression model...")
log_reg.fit(X_train, y_train)

y_pred_log_reg = log_reg.predict(X_test)
print(f"Training and Predicting Done!")

In [ ]:
# Evaluation
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg, average='weighted')
conf_mat_log_reg = confusion_matrix(y_test, y_pred_log_reg)

# Print Metrics
print("\nModel Performance and Evaluation")
print(f"Accuracy: {accuracy_log_reg:.4f}%")
print(f"F1 Score (Weighted): {f1_log_reg:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_log_reg)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_log_reg, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Logistic Regression')
plt.show()

In [ ]:
log_reg = LogisticRegression(max_iter=1000, solver='liblinear')

print(f"Training the Logistic Regression model...")
log_reg.fit(X_train, y_train)

y_pred_log_reg = log_reg.predict(X_test)
print(f"Training and Predicting Done!")

In [ ]:
# Evaluation
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg, average='weighted')
conf_mat_log_reg = confusion_matrix(y_test, y_pred_log_reg)

# Print Metrics
print("\nModel Performance and Evaluation")
print(f"Accuracy: {accuracy_log_reg:.4f}%")
print(f"F1 Score (Weighted): {f1_log_reg:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_log_reg)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_log_reg, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Logistic Regression')
plt.show()

In [ ]:
log_reg = LogisticRegression(C=10, max_iter=1000, solver='newton-cg')

print(f"Training the Logistic Regression model...")
log_reg.fit(X_train, y_train)

y_pred_log_reg = log_reg.predict(X_test)
print(f"Training and Predicting Done!")

In [ ]:
# Evaluation
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg, average='weighted')
conf_mat_log_reg = confusion_matrix(y_test, y_pred_log_reg)

# Print Metrics
print("\nModel Performance and Evaluation")
print(f"Accuracy: {accuracy_log_reg:.4f}%")
print(f"F1 Score (Weighted): {f1_log_reg:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_log_reg)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_log_reg, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Logistic Regression')
plt.show()

#### Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1 max_depth=3, random_state=42)

print(f"Training the Gradient Boosting model...")
gb.fit(X_train, y_train)

y_pred_gb = gb.predict(X_test)
print(f"Training and Predicting Done!")

In [ ]:
# Evaluation
accuracy_gb = accuracy_score(y_test, y_pred_gb)
f1_gb = f1_score(y_test, y_pred_gb, average='weighted')
conf_mat_gb = confusion_matrix(y_test, y_pred_gb)

# Print Metrics
print("\nModel Performance and Evaluation")
print(f"Accuracy: {accuracy_gb:.4f}%")
print(f"F1 Score (Weighted): {f1_gb:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_gb)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_gb, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Gradient Boosting')
plt.show()

#### XGBoost

In [ ]:
# xgboost = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
xgboost = xgb.XGBClassifier(scale_pos_weight=1, n_estimators=100, learning_rate=0.1, max_depth=3, gamma=0.1, eval_metric='logloss', random_state=42)

print(f"Training the XGBoost model...")
xgboost.fit(X_train, y_train)

y_pred_xgboost = xgboost.predict(X_test)
print(f"Training and Predicting Done!")

In [ ]:
# Evaluation
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost, average='weighted')
conf_mat_xgboost = confusion_matrix(y_test, y_pred_xgboost)

# Print Metrics
print("\nModel Performance and Evaluation")
print(f"Accuracy: {accuracy_xgboost:.4f}%")
print(f"F1 Score (Weighted): {f1_xgboost:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_xgboost)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_xgboost, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - XGBoost')
plt.show()

In [ ]:
# xgboost = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
xgboost = xgb.XGBClassifier(objective='multi:softmax', scale_pos_weight=1, n_estimators=100, learning_rate=0.1, max_depth=3, gamma=0.1, eval_metric='logloss', random_state=42)

print(f"Training the XGBoost model...")
xgboost.fit(X_train, y_train)

y_pred_xgboost = xgboost.predict(X_test)
print(f"Training and Predicting Done!")

In [ ]:
# Evaluation
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost, average='weighted')
conf_mat_xgboost = confusion_matrix(y_test, y_pred_xgboost)

# Print Metrics
print("\nModel Performance and Evaluation")
print(f"Accuracy: {accuracy_xgboost:.4f}%")
print(f"F1 Score (Weighted): {f1_xgboost:.4f}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_xgboost)}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_xgboost, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - XGBoost')
plt.show()